In [1]:
# Final Project Code:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

def convert_latitude(latitude_str):
    numerical_part = latitude_str.split('°')[0]  # Extract numerical part before '°'
    numerical_value = float(numerical_part)
    
    if 'S' in latitude_str:
        numerical_value = -numerical_value  # If latitude is in Southern hemisphere
    
    return numerical_value

def convert_longitude(longitude_str):
    numerical_part = longitude_str.split('°')[0]  # Extract numerical part before '°'
    numerical_value = float(numerical_part)
    
    if 'W' in longitude_str:
        numerical_value = -numerical_value  # If longitude is in Western hemisphere
    
    return numerical_value

In [10]:
file_path = 'MLS_Master_Sheet.XLSX'

# Read the Excel file into a DataFrame
df = pd.read_excel(file_path)

In [11]:
df['Latitude'] = df['Latitude'].apply(convert_latitude)
df['Longitude'] = df['Longitude'].apply(convert_longitude)

In [13]:
import plotly.graph_objects as go
import pandas as pd
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import time

 

# Reset index after groupby
df = df.groupby(['Year', 'Tm']).sum().reset_index()

 

# Create a Dash web application
app = dash.Dash(__name__)

 

# Define the app layout
app.layout = html.Div([
    dcc.Graph(id='animated-map'),
    dcc.Interval(
        id='interval-component',
        interval=1000,  # Update interval in milliseconds
        n_intervals=0
    )
])

 

@app.callback(
    Output('animated-map', 'figure'),
    Input('interval-component', 'n_intervals')
)
def update_map(n_intervals):
    year_index = n_intervals % len(df['Year'].unique())
    year = df['Year'].unique()[year_index]

 

    # Create an animated scatter plot using Plotly graph_objects
    fig = go.Figure()

 

    hover_text = []
    for index, row in df[df['Year'] == year].iterrows():
        champion_text = "Champion" if row['Champion'] == 1 else "Not Champion"
        text = f"Team: {row['Tm']}<br>Year: {year}<br>W-L%: {row['W-L%']}<br>{champion_text}"
        hover_text.append(text)

 

    scatter = go.Scattergeo(
        lon=df[df['Year'] == year]['Longitude'],
        lat=df[df['Year'] == year]['Latitude'],
        text=hover_text,  # Updated text property for hover information
        mode='markers',
        marker=dict(
            size=df[df['Year'] == year]['W-L%'] * 50,
            color=df[df['Year'] == year]['W-L%'],
            colorscale='Viridis',
            cmin=0.1,
            cmax=0.9,
            colorbar=dict(title='Win-Loss Percentage'),
        ),
        name=str(year),
    )
    fig.add_trace(scatter)

 

    fig.update_geos(
        scope='usa',
        showcoastlines=True,
        coastlinecolor="RebeccaPurple",
        showland=True,
        landcolor="LightGreen",
        showsubunits=True,
        subunitcolor="gray",
    )

 

    champion_team = df[(df['Year'] == year) & (df['Champion'] == 1)]['Tm'].values
    if champion_team:
        champion_text = f'Champion: {champion_team[0]}'
    else:
        champion_text = 'No Champion'

 

    fig.update_layout(
        title=f'MLS Win-Loss Percentage Over Time - {year}<br>{champion_text}',
        title_x=0.5  # Center the title
    )

 

    return fig

 

if __name__ == '__main__':
    app.run_server(debug=True, port = 8053)